In [ ]:
import os
import math
import numpy as np
import pandas as pd
import QuantLib as ql
from scipy.stats import norm
from scipy.special import iv
from scipy.integrate import quad
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')
import multiprocessing
import logging
logging.getLogger().setLevel(logging.DEBUG)

### Price calculation for option (S_{T2}-S_{T1})^+ with Monte Carlo method and analytic formula

In [ ]:
def price_MonteCarlo(params, S0):
    '''
    Calculate price of option (S2-S1)^+ with Monte Carlo method
    params: a list contains V0, kappa, theta, sigma, rho, r, t, T1, T2
    V0, kappa, theta, sigma, rho : params in heston model
    r: interest rate at current time
    t: current time from t0, float
    T1, T2: two maturities from t0 in payoff, float
    '''
    N = 1000
    M = 50000
    V0, kappa, theta, sigma, rho, r, t, T1, T2 = params

    # Simulation time settings
    dt = T2 / N  # Time increment

    # Correlated random numbers
    W1 = np.random.normal(0, 1, (M, N))
    Z2 = np.random.normal(0, 1, (M, N))
    W2 = rho * W1 + np.sqrt(1 - rho ** 2) * Z2

    # Initialize stock price and variance arrays
    S = np.zeros((M, N + 1))
    V = np.zeros((M, N + 1))
    S[:, 0] = S0
    V[:, 0] = V0

    # Simulate paths
    for t in range(1, N + 1):
        # Ensure variance remains positive
        V[:, t] = np.maximum(
            V[:, t - 1] + kappa * (theta - V[:, t - 1]) * dt + sigma * np.sqrt(V[:, t - 1] * dt) * W2[:, t - 1], 0)
        S[:, t] = S[:, t - 1] * np.exp((r - 0.5 * V[:, t - 1]) * dt + np.sqrt(V[:, t - 1] * dt) * W1[:, t - 1])
    if T1 == 0:
        payoff_simu = calc_Payoff(S0, S[:, int(T2 / dt)])
    else:
        # Get values at T1 and T2
        S1 = S[:, int(T1 / dt)]
        S2 = S[:, int(T2 / dt)]

        # Calculate payoff and discount it back to present value
        payoff_simu = calc_Payoff(S1, S2)
    return np.mean(payoff_simu) * np.exp(r * T2 / 252)

## The following functions are intermidate elements for Heston model analytic price (Kruse2005)

# Define C_j(φ, T - t)
def C_j(phi, T_t, b_j, d_j, q_j, kappa, theta, sigma, rho, r):
    I = 1j  # sqrt(-1)
    term1 = r * I * phi * T_t
    term2 = (kappa * theta / sigma**2) * ((b_j - rho * sigma * I * phi + d_j) * T_t)
    try:
        term3 = -2 * (kappa * theta / sigma**2) * (np.log(1 - q_j) + (d_j * T_t) - np.log(1 - q_j))
    except RuntimeWarning as e:
        print(f"Caught a RuntimeWarning: {e}")
    return term1 + term2 + term3

# Define D_j(φ, T - t)
def D_j(phi, T_t, b_j, d_j, q_j, sigma, rho):
    I = 1j  # sqrt(-1)
    numerator = b_j - rho * sigma * I * phi + d_j
    denominator = sigma**2
    term1 = numerator / denominator
    try:
        temp = np.exp(d_j * T_t)
        term2 = np.log(1 - temp) - np.log(1 - q_j * temp)
    except:
        return term1 / q_j
    return term1 * np.exp(term2)


# Define the characteristic function f_j
def f_j(j, S_t, v_t, T_t, phi, kappa, theta, sigma, rho, r):
    I = 1j  # Define sqrt(-1) as the imaginary unit
    ln_S_t = np.log(S_t)

    if j == 1:
        b_j = kappa - sigma * rho
        u_j = 1/2
    elif j == 2:
        b_j = kappa + sigma * rho
        u_j = -1/2
    d_j = np.sqrt((rho * sigma * I * phi - b_j)**2 - sigma**2 * (2 * u_j * I * phi - phi**2))
    q_j = (b_j - rho * sigma * I * phi + d_j) / (b_j - rho * sigma * I * phi - d_j)
    result = np.exp(I * phi * ln_S_t + C_j(phi, T_t, b_j, d_j, q_j, kappa, theta, sigma, rho, r) + D_j(phi, T_t, b_j, d_j, q_j, sigma, rho) * v_t)
    return result

# Define the main function P_j
def P_j(t, S_t, v_t, j, T, kappa, theta, sigma, rho, r, K=1):
    # Define the integrand
    def integrand(phi):
        I = 1j  # sqrt(-1)
        f_j_value = f_j(j, S_t, v_t, T - t, phi, kappa, theta, sigma, rho, r)
        return max(np.real(np.exp(-I * phi * np.log(K)) * f_j_value / (I * phi)),1e6)

    integral_value, _ = quad(integrand, 0, np.inf,limit=100000)
    # Combine the components of P_j
    P_value = 0.5 + (1 / np.pi) * integral_value
    return max(min(P_value,1e2),-1e2)

# Define the probability density function f(v|ν(t))
def f_v_given_vega(t, t_star, vega, kappa, theta, sigma, rho):
    B = (4 * kappa * (1 - np.exp(-(kappa - rho * sigma) * (t_star - t)))) / sigma**2
    R = 4 * kappa * theta / sigma**2
    Lambda = B * np.exp(-(kappa - rho * sigma) * (t_star - t)) * vega
    term1 = B / 2
    term2 = np.exp(min(700,-(B * vega + Lambda)) / 2)
    try:
        term3 = (B * vega / Lambda) ** ((R / 2 - 1) / 2)
        term4 = iv(R / 2 - 1, np.sqrt(Lambda * B * vega))  # Modified Bessel function
        result = term1 * term2 * term3 * term4
    except:
        return 0
    return max(min(result,1e5),-1e5)


# Define the integral of P̂_j
def P_hat_j(S_t, j, params):
    vega_t, kappa, theta, sigma, rho, r, t, t_star, T = params
    def integrand(v):
        inte_p = P_j(t_star, S_t, v, j, T, kappa, theta, sigma, rho, r)
        inte_f = f_v_given_vega(t, t_star, v, kappa, theta, sigma, rho)
        return inte_p * inte_f
    integral_value, _ = quad(integrand, 0, np.inf)
    return min(max(integral_value,0),1)

def C_FWS(S_t, params,returnP=0):
    '''
    Analytic pricing under Heston model(Kruse2005)
    '''
    vega_t, kappa, theta, sigma, rho,r, t, t_star, T = params
    P_hat_1 = P_hat_j(S_t, 1, params)
    P_hat_2 = P_hat_j(S_t, 2, params)
    term1 = S_t * P_hat_1
    term2 = np.exp(-r * (t_star - t)) * P_hat_2
    if returnP:
        return term1 - term2, P_hat_1, P_hat_2
    else:
        return term1 - term2

### Heston model parameter calibration with Quantlib and tikhonov regularization

In [ ]:
# Define a helper to price options using the Heston model
def model_price(strike, expiry, model):
    payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike)
    exercise = ql.EuropeanExercise(expiry)
    option = ql.VanillaOption(payoff, exercise)
    engine = ql.AnalyticHestonEngine(model)
    if engine is None:
        print("Pricing engine is not initialized properly.")
    option.setPricingEngine(engine)
    return option.NPV()


# Define the objective function (least squares + Tikhonov regularization)
def objective_function(params, market_data, alpha, prior_params):
    v0, kappa, theta, sigma, rho = params

    strikes = market_data['strike_prices']
    market_prices = market_data['market_prices']
    expiries = market_data['expiry']
    rf = market_data['risk_free_curve']

    heston_process = ql.HestonProcess(ql.YieldTermStructureHandle(rf),
                                      market_data['dividend_yield'], market_data['spot_price'],
                                      v0, kappa, theta, sigma, rho)
    heston_model = ql.HestonModel(heston_process)

    # Calculate the sum of squared errors between model and market prices
    errors = np.sum([(model_price(strike, expiry, heston_model) - market_price) ** 2
                     for strike, market_price, expiry in zip(strikes, market_prices, expiries)])

    # Tikhonov regularization (penalty for deviating from prior guess)
    regularization = alpha * np.sum((params - prior_params) ** 2)

    return errors + regularization


# Calibrate the parameters on each t0
def calibrateHestonT0(df, t0, S0, r0, init_params):
    valuation_date = t0
    ql.Settings.instance().evaluationDate = valuation_date
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(S0))
    dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, 0.0, day_count))
    risk_free_curve = ql.FlatForward(valuation_date, ql.QuoteHandle(ql.SimpleQuote(r0)), day_count)

    market_data = {
        'strike_prices': df['K'],
        'market_prices': (df['ask'] + df['bid']) / 2,  # Observed market option prices
        'expiry': [date_asQuantLib(t) for t in df['t']],
        'spot_price': spot_handle,
        'dividend_yield': dividend_yield,
        'risk_free_curve': risk_free_curve
    }

    initial_guess = init_params
    prior_params = init_params  # Prior guess for Tikhonov
    alpha = 0.01  # Regularization parameter

    result = minimize(
        objective_function,
        x0=initial_guess,
        args=(market_data, alpha, prior_params),
        method='L-BFGS-B',  # Optimization method, can use other methods
        bounds=[(0.01, 0.2), (0.1, 10), (0.001, 0.2), (0.001, 10.0), (-0.9, 0.9)]  # Bounds on parameters
    )

    return result.x
    
def t0_main(t0, data, stock_df, init_params):
    '''
    Calibrate all Heston params in each t0 with daily data
    '''
    print(t0)
    df = data[data['t0'] == t0]
    df = df[df['t'] > t0]
    S0 = stock_df.loc[t0]['adjusted_price']
    r = IR_effect(t0, t0, t0)
    params = calibrateHestonT0(df, date_asQuantLib(t0), S0, r, init_params)
    temp = pd.DataFrame(params, columns=pd.MultiIndex.from_tuples([(ticker, t0)])).T
    temp.columns = ['v0', 'kappa', 'theta', 'sigma', 'rho']
    temp.to_csv(f'params/temps/params_{t0}_{ticker}.csv')
    return temp

In [ ]:
v0 = 0.04  # Initial variance
kappa = 0.8  # Rate of mean reversion
theta = 0.04  # Long-term variance
sigma = 0.1  # Volatility of volatility
rho = -0.2  # Correlation (can be negative)
init_params = np.array([v0, kappa, theta, sigma, rho])

IRParams = pd.read_csv('interest_rates_parameters.csv', parse_dates=['Date'], dayfirst=True)
IRParams = IRParams.fillna(method='ffill').sort_values(by='Date')
IRParams.head()

calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
day_count = ql.Actual365Fixed()

tickers = ['JNJ','AMZN','GOOGL','JPM','MSFT','PG','TSLA','V','WMT']

params_all = pd.read_csv('hestonParams1.csv', index_col=[0,1])
ticker = 'MSFT'
stock_df = pd.read_csv('data/adjusted_Stock_Daily/{}_stock_daily_adjusted.csv'.format(ticker.lower()), index_col=0)
stock_df = stock_df.set_index('date')

data = pd.read_csv(f'rawImpl_localVol_{ticker.upper()}.csv',index_col=0)
data.columns = ['t0', 't', 'K', 'ask', 'bid','impl_volatility']
data['mid_price'] = data[['ask','bid']].mean(axis=1)
idx = params_all.loc[ticker].index
t0List = data['t0'].unique()
t0List = set(t0List) - set(idx)
    
for t0 in t0List:
    t0_main(t0, data, stock_df, init_params)

### Daily Delta-Hedging with Heston model

In [ ]:
nasdaq_holidays = [
    '2018-01-01', '2018-01-15', '2018-02-19', '2018-03-30', '2018-05-28',
    '2018-07-04', '2018-09-03', '2018-11-22', '2018-12-25',
    '2019-01-01', '2019-01-21', '2019-02-18', '2019-04-19', '2019-05-27',
    '2019-07-04', '2019-09-02', '2019-11-28', '2019-12-25',
    '2020-01-01', '2020-01-20', '2020-02-17', '2020-04-10', '2020-05-25',
    '2020-07-03', '2020-09-07', '2020-11-26', '2020-12-25',
    '2021-01-01', '2021-01-18', '2021-02-15', '2021-04-02', '2021-05-31',
    '2021-07-05', '2021-09-06', '2021-11-25', '2021-12-24',
    '2022-01-01', '2022-01-17', '2022-02-21', '2022-04-15', '2022-05-30',
    '2022-07-04', '2022-09-05', '2022-11-24', '2022-12-26',
    '2023-01-01', '2023-01-16', '2023-02-20', '2023-04-07', '2023-05-29',
    '2023-07-04', '2023-09-04', '2023-11-23', '2023-12-25'
]
nasdaq_holidays = np.array(nasdaq_holidays, dtype='datetime64[D]')


def IR_effect(init_date, start_date, end_date):
    if start_date == end_date:
        return 0.0
    else:
        tau = np.busday_count(start_date, end_date, holidays=nasdaq_holidays) / 252
        if tau == 0.0:
            return 0.0
        else:
            beta0 = IRParams.loc[IRParams.Date <= init_date, 'BETA0'].values[-1]
            beta1 = IRParams.loc[IRParams.Date <= init_date, 'BETA1'].values[-1]
            beta2 = IRParams.loc[IRParams.Date <= init_date, 'BETA2'].values[-1]
            tau1 = IRParams.loc[IRParams.Date <= init_date, 'TAU1'].values[-1]

            r = beta0 + beta1*(1-math.exp(-tau/tau1))/(tau/tau1) + beta2*((1-math.exp(-tau/tau1))/(tau/tau1)-math.exp(-tau/tau1))
            return r/100


def date_asQuantLib(t):
    result = pd.to_datetime(t)
    return ql.Date(result.day, result.month, result.year)


def read_all_paths(directory):
    all_paths = []
    for root, dirs, files in os.walk(directory):
        for name in files:
            file_path = os.path.join(root, name)
            all_paths.append(file_path)
        for name in dirs:
            dir_path = os.path.join(root, name)
            all_paths.append(dir_path)
    return all_paths

def calc_Payoff(S1, S2):
    return np.maximum(S2 - S1, 0)

def calc_GeekingHedge(S0, params, alpha):
    delta = C_FWS(S0*(1+alpha), params) - C_FWS(S0, params)
    return delta / (alpha * S0)

def Heston_main(item):
    '''
    Main function for heston daily delta-hedging, using params calibrated above
    When the heston possibilities in analytic formula is out of reasonable range, Monte Carlo method is used
    '''
    pair, t0List = item
    print(pair)

    t1 = pd.to_datetime(pair.split('_')[1]).strftime('%Y-%m-%d')
    t2 = pd.to_datetime(pair.split('_')[2]).strftime('%Y-%m-%d')
    ticker = pair.split('_')[0].lower()
    stock_df = pd.read_csv('data/adjusted_Stock_Daily/{}_stock_daily_adjusted.csv'.format(ticker.lower()), index_col=0)
    stock_df = stock_df.set_index('date')

    params_df = pd.read_csv('hestonParams1.csv',index_col=[0,1]).loc[ticker.upper()]

    k = 1
    t0List.sort()
    gap_list = []
    for i in range(len(t0List)-1,-1,-1):
        t0 = t0List[i]
        S0 = stock_df.loc[t0]['adjusted_price']
        r = IR_effect(t0, t0, t2)
        payoff_Actual = calc_Payoff(stock_df.loc[t1, 'adjusted_price'], stock_df.loc[t2, 'adjusted_price'])

        T1 = day_count.yearFraction(date_asQuantLib(t0), date_asQuantLib(t1))
        T2 = day_count.yearFraction(date_asQuantLib(t0), date_asQuantLib(t2))
        params = np.array(params_df.loc[t0])
        params = np.concatenate((params, np.array([r,0,T1,T2])))
        price, P_1, P_2 = C_FWS(S0, params,returnP=1)
        if math.isnan(price) or P_1 == 0 or P_1 == 1:
            price = price_MonteCarlo(params, S0)
        if math.isnan(price):
            gap_list.append(np.nan)
            continue
        t_i = pd.to_datetime(t0)
        while t_i < pd.to_datetime(t2):
            ti_str = t_i.strftime('%Y-%m-%d')
            S_i = stock_df.loc[ti_str:].iloc[0, 0]
            r = IR_effect(ti_str, ti_str, t2)
            t = max([date for date in t0List if date <= ti_str])
            params_hedge = np.array(params_df.loc[t])
            params_hedge = np.concatenate((params_hedge, np.array([r, day_count.yearFraction(date_asQuantLib(t_i), date_asQuantLib(t)),T1,T2])))
            if t_i <= pd.to_datetime(t1):
                delta_i = calc_GeekingHedge(S_i, params_hedge, alpha=0.001)
            else:
                K = stock_df.loc[t1, 'adjusted_price']
                sigma = params_hedge[3]
                d1 = (np.log(S_i / K) + (r + 0.5 * sigma**2) * 2) / (sigma * np.sqrt(2))
                delta_i = norm.cdf(d1)
            delta_i = 0 if math.isnan(delta_i) else delta_i
            t_i += pd.Timedelta(days=1)
            T1 = day_count.yearFraction(date_asQuantLib(t_i), date_asQuantLib(t1))
            T2 = day_count.yearFraction(date_asQuantLib(t_i), date_asQuantLib(t2))
            price += delta_i * (stock_df.loc[:t_i.strftime('%Y-%m-%d')].iloc[-1, 0] - S_i)
        gap_list.append((price - payoff_Actual) / S0)
    timeToMaturity = np.busday_count(pd.to_datetime(t0List[::-1]).values.astype('datetime64[D]'),
                                     pd.to_datetime([t1]).values.astype('datetime64[D]'),
                                     holidays=nasdaq_holidays)
    gap_list = pd.DataFrame(gap_list, index=timeToMaturity, columns=[pair]).T
    gap_list.to_csv('result_Heston/{}.csv'.format(pair))
    return gap_list

In [ ]:
if __name__ == "__main__":
    IRParams = pd.read_csv('interest_rates_parameters.csv', parse_dates=['Date'], dayfirst=True)
    IRParams = IRParams.fillna(method='ffill').sort_values(by='Date')
    IRParams.head()

    calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
    day_count = ql.Actual365Fixed()

    data_path = f'data/options_call_askbid/'
    tickerList = ['JNJ','AMZN','GOOGL','JPM','MSFT','PG','TSLA','V','WMT']

    paths = os.listdir(data_path)
    paths = [file for file in paths if file[-4:] == '.csv' and file.split('_')[0] in tickerList]
    gap_df = pd.read_csv('result_hestonModel.csv',index_col=0)
    gap_df.columns = np.array(pd.to_numeric(gap_df.columns).astype('int'))
    data_dict = {}
    for path in paths:
        data = pd.read_csv(data_path+path, index_col=0)
        pair = path.split('.')[0].split('/')[-1]
        if pair not in gap_df.index:
            t0List = data['t0'].unique()
            data_dict[pair] = t0List

    manager = multiprocessing.Manager()
    results = manager.list()

    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        for item in data_dict.items():
            pool.apply_async(Heston_main, args=(item,), callback=collect_results)

        pool.close()
        pool.join()

    combined_result = pd.concat(results)

    combined_result.to_csv('result_Heston.csv')
    logging.info("All tasks completed and results saved to result.csv.")